In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot
import nltk
import string
import os
import re

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')

[nltk_data] Downloading package stopwords to /home/denis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/denis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/denis/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [2]:
df = pd.read_json('../data/data.json')
print(df.count())
df.head()

id                 99109
executable_name    99109
browser_title      99109
dtype: int64


,id,executable_name,browser_title
0,1,3D Hamster,????? ????? ???????? ???????
1,2,3D Hamster,3D Hamster v.0.6.2
2,3,3ds Max 2021,3dsmax
3,4,3ds Max 2021,Data collection and use
4,5,3ds Max 2021,Unbenannt - Autodesk 3ds Max 2021


In [3]:
df = df.drop_duplicates(subset=['executable_name', 'browser_title'])
df.to_csv('../data/unique.csv', index=False)
df.count()

id                 99109
executable_name    99109
browser_title      99109
dtype: int64

In [6]:
bag_of_words: dict[str, bool] = {}

In [4]:
df['browser_title'] = df['browser_title'].apply(lambda x: re.sub(r'Telegram \([0-9]+\)', 'Telegram', x))
df['browser_title'] = df['browser_title'].apply(lambda x: re.sub(r'.*Twitter.*', 'Twitter', x))
df['browser_title'] = df['browser_title'].apply(lambda x: re.sub(r'.*Твиттер.*', 'Twitter', x))
df['Category'] = ''
df = df.drop_duplicates(subset=['browser_title'])
df

,id,executable_name,browser_title,Category
0,1,3D Hamster,????? ????? ???????? ???????,
1,2,3D Hamster,3D Hamster v.0.6.2,
2,3,3ds Max 2021,3dsmax,
3,4,3ds Max 2021,Data collection and use,
4,5,3ds Max 2021,Unbenannt - Autodesk 3ds Max 2021,
...,...,...,...,...
99096,99097,Хост-процесс Windows (Rundll32),Настройка печати: 470_Kyocera_3645 на fs,
99098,99099,Хост-процесс Windows (Rundll32),Оповещение системы безопасности Windows,
99099,99100,Хост-процесс Windows (Rundll32),Переменные среды,
99101,99102,Хост-процесс Windows (Rundll32),Свойства: 470_Kyocera_3645 на fs,


In [5]:
blacklist = ['фенибут', "секс", 'porn', 'pornstar', 'anal']

In [7]:
# corp = nltk.corpus.words.words()
def count_words(data: pd.DataFrame):
    def counter(row):
        if row['browser_title'] == '':
            words = row['executable_name'].split()
        else:
            words = list(set(row['executable_name'].split() + row['browser_title'].split()))
        count = 0
        for word in words:
            if word.lower() in blacklist:
                return -1
            if word not in bag_of_words:
                bag_of_words[word] = True
                count += 1
        return count
    bag_of_words = {}
    return data.apply(counter, axis=1)

In [8]:
df['count'] = count_words(df)
df

,id,executable_name,browser_title,Category,count
0,1,3D Hamster,????? ????? ???????? ???????,,5
1,2,3D Hamster,3D Hamster v.0.6.2,,1
2,3,3ds Max 2021,3dsmax,,4
3,4,3ds Max 2021,Data collection and use,,4
4,5,3ds Max 2021,Unbenannt - Autodesk 3ds Max 2021,,3
...,...,...,...,...,...
99096,99097,Хост-процесс Windows (Rundll32),Настройка печати: 470_Kyocera_3645 на fs,,1
99098,99099,Хост-процесс Windows (Rundll32),Оповещение системы безопасности Windows,,0
99099,99100,Хост-процесс Windows (Rundll32),Переменные среды,,0
99101,99102,Хост-процесс Windows (Rundll32),Свойства: 470_Kyocera_3645 на fs,,0


In [9]:
df.sort_values(by='count', ascending=False)

,id,executable_name,browser_title,Category,count
21776,21744,Google Chrome,'charmap' codec can't decode byte 0x98 in posi...,,52
30933,30869,Google Chrome,"In Fig. 10 - 50, two 6,20 kg blocks are connec...",,30
5155,5140,chrome,[[Publication Title: software] OR [Publication...,,28
30884,30820,Google Chrome,"#include < class Point { private double m_x, m...",,27
11789,11754,chrome.exe,"номер, регистратура, запись, Городская поликли...",,27
...,...,...,...,...,...
55604,55406,Google Chrome,Что бесит работника секс шопа | Стас Ярков - Y...,,-1
53613,53432,Google Chrome,Секс в авто Казань - Google Chrome,,-1
53614,53433,Google Chrome,Секс Знакомства Интим Казань в Казани - Google...,,-1
53615,53434,Google Chrome,"Секс игрушка в виде черного лисьего хвоста, со...",,-1


In [10]:
df_2: pd.DataFrame = df.iloc[:12000].sort_values(by='count', ascending=False).drop(columns=['count'])
df_2

,id,executable_name,browser_title,Category
5155,5140,chrome,[[Publication Title: software] OR [Publication...,
11789,11754,chrome.exe,"номер, регистратура, запись, Городская поликли...",
11370,11340,chrome.exe,И повсюду тлеют пожары / Little Fires Everywhe...,
8578,8554,chrome.exe,"Kiabi.ru, магазин одежды для женщин, мужчин, д...",
4404,4390,chrome,IamMaxim/vigilant-palm-tree: Why Vigilant Palm...,
...,...,...,...,...
858,855,Adobe Acrobat Reader DC,Introduction - Adobe Acrobat Reader DC,
9324,9298,chrome.exe,Personal computing: personal computers and the...,
9326,9300,chrome.exe,pgAdmin 4 - Google Chrome,
4198,4184,chrome,for - JavaScript | MDN - Google Chrome,


In [13]:
df_2.to_csv('../data/unique.csv', index=False, sep='\t')

In [16]:
df_3 = pd.read_csv('../data/saved.csv', sep='\t')
df_3.set_index('id', inplace=True)
df_3

,Category
id,
5140,Development
11754,Utilities
11340,Entertainment
8554,Utilities
4390,Development
...,...
855,Utilities
9298,NaN
9300,NaN


In [14]:
df_2.set_index('id', inplace=True)
df_2

,executable_name,browser_title,Category
id,,,
5140,chrome,[[Publication Title: software] OR [Publication...,
11754,chrome.exe,"номер, регистратура, запись, Городская поликли...",
11340,chrome.exe,И повсюду тлеют пожары / Little Fires Everywhe...,
8554,chrome.exe,"Kiabi.ru, магазин одежды для женщин, мужчин, д...",
4390,chrome,IamMaxim/vigilant-palm-tree: Why Vigilant Palm...,
...,...,...,...
855,Adobe Acrobat Reader DC,Introduction - Adobe Acrobat Reader DC,
9298,chrome.exe,Personal computing: personal computers and the...,
9300,chrome.exe,pgAdmin 4 - Google Chrome,


In [17]:
df_2.drop('Category', axis=1, inplace=True)
df_2

,executable_name,browser_title
id,,
5140,chrome,[[Publication Title: software] OR [Publication...
11754,chrome.exe,"номер, регистратура, запись, Городская поликли..."
11340,chrome.exe,И повсюду тлеют пожары / Little Fires Everywhe...
8554,chrome.exe,"Kiabi.ru, магазин одежды для женщин, мужчин, д..."
4390,chrome,IamMaxim/vigilant-palm-tree: Why Vigilant Palm...
...,...,...
855,Adobe Acrobat Reader DC,Introduction - Adobe Acrobat Reader DC
9298,chrome.exe,Personal computing: personal computers and the...
9300,chrome.exe,pgAdmin 4 - Google Chrome


In [18]:
df_2['Category'] = df_3['Category']
df_2

,executable_name,browser_title,Category
id,,,
5140,chrome,[[Publication Title: software] OR [Publication...,Development
11754,chrome.exe,"номер, регистратура, запись, Городская поликли...",Utilities
11340,chrome.exe,И повсюду тлеют пожары / Little Fires Everywhe...,Entertainment
8554,chrome.exe,"Kiabi.ru, магазин одежды для женщин, мужчин, д...",Utilities
4390,chrome,IamMaxim/vigilant-palm-tree: Why Vigilant Palm...,Development
...,...,...,...
855,Adobe Acrobat Reader DC,Introduction - Adobe Acrobat Reader DC,Utilities
9298,chrome.exe,Personal computing: personal computers and the...,NaN
9300,chrome.exe,pgAdmin 4 - Google Chrome,NaN


In [20]:
df_2.to_csv('../data/tagged_true.csv', sep='\t')

In [79]:
# save_by_parts(df_2)

In [7]:
def preprocess(text):
    """
    Remove all punctuation and stopwords and returns a list of the cleaned text
    """
    nopunc = [char for char in text if char not in string.punctuation]

    nopunc = ''.join(nopunc).lower()

    return ' '.join([word for word in nopunc.split() if word.lower() not in nltk.corpus.stopwords.words('english') and word.isalpha()])
df["browser_title"] = df["browser_title"].apply(preprocess)
df.head()

,id,executable_name,browser_title
0,4174836,Проводник,
1,4174903,Блокнот,дерево
2,4174904,Google Chrome,lab google chrome
3,4174774,WinRAR archiver,
4,4174962,Google Chrome,karinatyulebaevadatabaselabs google chrome


In [ ]:
df_2